# Implementierung eines Reinforcement-Learning-Projekts

## Das Problem
Wir sind Betreiber eine Spedition und sind zuständig für die Belieferung von Supermärkten. Dafür muss Ware zwischen Lagern und den Märkten innerhalb unserer Stadt transportiert werden. Unser innovativer LKW ist selbstfahrend und bekommt lediglich den Auftrag an einer Station (Lager oder Supermarkt) Ware einzusammeln und an einer anderen Station wieder abzuliefern.

## Definitionen
Wir bedienen zwei Lager und zwei Supermärkte.

Lager:
0. A-Lager (A)
1. B-Lager (B)

Supermarkt:
2. C-Markt (C)
3. D-Markt (D)

Diese verteilen sich folgendermaßen in unserer Stadt:

<img src="img/stadt.png" alt="stadt" width="400"/>

Die Stadt ist eine 6x6-Quadratestadt und mit 36 Positionen, die die Koordinaten $(0,0)$ bis $(5,5)$ haben. Der LKW kann sich frei in der Stadt bewegen, aber nicht durch die Grünstreifen fahren.

Die Anzahl der Zustände ergibt sich folgendermaßen:
* 6 x 6 Positionen
* 4 Orte zu denen die Ware gebracht werden kann (A bis D bzw. 0 bis 3)
* 5 Orte, an denen sich die Ware befindet (A bis D bzw. 0 bis 3 und im LKW (Position Nr.4))

\\[ 6 \cdot 6 \cdot 4 \cdot 5 = 720 \texttt{ mögliche Zustände}\\]

Die Aktionen, die der LKW ausführen kann sind:
0. nach Norden fahren
1. nach Osten fahren
2. nach Süden fahren
3. nach Westen fahren
4. Ware einsammeln
5. Ware abladen

Dabei kann er folgende Belohnungen (und Abzüge) erhalten:
* Ware korrekt abliefern: +20
* Ware falsch einsammeln/abliefern: -10
* Pro Schritt: -1

## Definitionen implementieren

In [ ]:
city = [
    "+-----------+",
    "|A: : |B: : |",
    "|_: : | : :_|",
    "| : : : : : |",
    "| : | : :_:_|",
    "| | : : : :D|",
    "|C| : | : : |",
    "+-----------+",
]
num_rows = 6
num_cols = 6
rows = [row for row in range(0, num_rows)]
cols = [col for col in range(0, num_cols)]
num_actions = 6
stations = [(0,0), (0,3), (5,0), (4,5)]